In [2]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.5 MB/s eta 0:00:00


In [7]:
# Import library
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re
import numpy as np

# 1. Membaca dataset JSON
data = pd.read_json('berita.json')

# Menampilkan preview data
print("Step 1: Preview data awal:")
print(data[['title', 'content']].head())

# 2. Preprocessing
def preprocess_text(text):
    # Lowercasing
    text = text.lower()

    # Menghapus karakter non-alfabet
    text = re.sub(r'[^a-z\s]', '', text)

    # Tokenisasi
    tokens = text.split()

    # Stopword removal dan stemming
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Daftar stopword
    stopwords = set(["yang", "dan", "di", "ke", "dari", "pada", "untuk", "dengan"])
    tokens = [stemmer.stem(word) for word in tokens if word not in stopwords]

    # Gabungkan kembali token
    return ' '.join(tokens)

print("\nStep 2: Proses Preprocessing - Sebelum dan Sesudah:")
for idx, row in data.head(5).iterrows():
    original = row['content']
    processed = preprocess_text(original)
    print(f"Original: {original[:100]}...")
    print(f"Processed: {processed[:100]}...")
    print("")

# Terapkan preprocessing pada kolom 'content'
data['processed_content'] = data['content'].apply(preprocess_text)

# 3. Vektorisasi dengan TF-IDF
print("\nStep 3: TF-IDF Vektorisasi - Preview Matriks:")
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data['processed_content'])

# Menampilkan sebagian kecil hasil TF-IDF
feature_names = vectorizer.get_feature_names_out()
dense_matrix = tfidf_matrix.todense()
sample_matrix = pd.DataFrame(dense_matrix[:5, :5], columns=feature_names[:5])
print(sample_matrix)

# 4. Fungsi rekomendasi
def recommend_articles(input_title, data, tfidf_matrix, top_n=5):
    # Cari index artikel berdasarkan judul
    idx = data[data['title'] == input_title].index[0]

    # Hitung cosine similarity
    cosine_sim = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()

    # Urutkan berdasarkan skor similarity
    similar_indices = cosine_sim.argsort()[::-1][1:top_n+1]

    # Ambil artikel dengan skor tertinggi
    recommended = data.iloc[similar_indices].copy()
    recommended['similarity'] = cosine_sim[similar_indices]

    # Tambahkan filter kategori jika diperlukan
    category = data.iloc[idx]['category']
    recommended = recommended[recommended['category'] == category]

    return recommended

# 5. Pilihan artikel
print("\nStep 5: Pilih artikel berdasarkan nomor:")
for i, title in enumerate(data['title']):
    print(f"{i + 1}. {title}")

# Ambil input dari pengguna
try:
    choice = int(input("\nMasukkan nomor artikel pilihan: ")) - 1
    if 0 <= choice < len(data):
        input_article = data.iloc[choice]['title']
        recommendations = recommend_articles(input_article, data, tfidf_matrix)
        print(f"\nRekomendasi untuk artikel: {input_article}")
        print(recommendations[['title', 'similarity']])
    else:
        print("Nomor yang dimasukkan tidak valid.")
except ValueError:
    print("Input harus berupa angka.")

Step 1: Preview data awal:
                                               title  \
0  Honda Vario 125 Lama Banyak Dicari, Waspada Pe...   
1  Sering Disangka TPS Error, Ini Gejala Fuel Pum...   
2  Begini Cara Mudah Bikin Tarikan Motor Matic Ho...   
3  Aerox 155 dan Vario 125 Korban Tebas Gran Max ...   
4  Lebih Murah, Subtitusi Rantai Keteng Vario 125...   

                                             content  
0  Honda Vario 125 generasi pertama hingga sekara...  
1  Berikut beberapa gejala yang bisa dirasakan ji...  
2  Tarikan motor matic Honda Vario 125 kesayangan...  
3  Ajal menjemput mas Suraji (39) pengendaran Yam...  
4  Rantai keteng berfungsi sebagai penghubung ant...  

Step 2: Proses Preprocessing - Sebelum dan Sesudah:
Original: Honda Vario 125 generasi pertama hingga sekarang masih banyak dicari untuk bahan modifikasi, ataupun...
Processed: honda vario generasi pertama hingga sekarang masih banyak cari bahan modifikasi atau pakai hari jika...

Original: Berikut beber

## INTERAKTIF

In [3]:
# Import library
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re
import numpy as np

# 1. Membaca dataset JSON
data = pd.read_json('berita.json')

# Menampilkan preview data
print("Preview data:")
print(data.head())

# 2. Preprocessing
def preprocess_text(text):
    # Lowercasing
    text = text.lower()

    # Menghapus karakter non-alfabet
    text = re.sub(r'[^a-z\s]', '', text)

    # Tokenisasi
    tokens = text.split()

    # Stopword removal dan stemming
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Daftar stopword
    stopwords = set(["yang", "dan", "di", "ke", "dari", "pada", "untuk", "dengan"])
    tokens = [stemmer.stem(word) for word in tokens if word not in stopwords]

    # Gabungkan kembali token
    return ' '.join(tokens)

# Terapkan preprocessing pada kolom 'content'
data['processed_content'] = data['content'].apply(preprocess_text)

# 3. Vektorisasi dengan TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data['processed_content'])

# 4. Fungsi rekomendasi
def recommend_articles(input_title, data, tfidf_matrix, top_n=5):
    # Cari index artikel berdasarkan judul
    idx = data[data['title'] == input_title].index[0]

    # Hitung cosine similarity
    cosine_sim = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()

    # Urutkan berdasarkan skor similarity
    similar_indices = cosine_sim.argsort()[::-1][1:top_n+1]

    # Ambil artikel dengan skor tertinggi
    recommended = data.iloc[similar_indices].copy()
    recommended['similarity'] = cosine_sim[similar_indices]

    # Tambahkan filter kategori jika diperlukan
    category = data.iloc[idx]['category']
    recommended = recommended[recommended['category'] == category]

    return recommended

# 5. Contoh penggunaan
input_article = "Honda Vario 125 Lama Banyak Dicari, Waspada Penyakit Ini Kalau Berniat Beli"

if input_article in data['title'].values:
    recommendations = recommend_articles(input_article, data, tfidf_matrix)
    print(f"Rekomendasi untuk artikel: {input_article}")
    print(recommendations[['title', 'similarity']])
else:
    print(f"Artikel '{input_article}' tidak ditemukan di dataset.")

Preview data:
                                               title  \
0  Honda Vario 125 Lama Banyak Dicari, Waspada Pe...   
1  Sering Disangka TPS Error, Ini Gejala Fuel Pum...   
2  Begini Cara Mudah Bikin Tarikan Motor Matic Ho...   
3  Aerox 155 dan Vario 125 Korban Tebas Gran Max ...   
4  Lebih Murah, Subtitusi Rantai Keteng Vario 125...   

                                             content  category  
0  Honda Vario 125 generasi pertama hingga sekara...  Otomotif  
1  Berikut beberapa gejala yang bisa dirasakan ji...  Otomotif  
2  Tarikan motor matic Honda Vario 125 kesayangan...  Otomotif  
3  Ajal menjemput mas Suraji (39) pengendaran Yam...  Otomotif  
4  Rantai keteng berfungsi sebagai penghubung ant...  Otomotif  
Rekomendasi untuk artikel: Honda Vario 125 Lama Banyak Dicari, Waspada Penyakit Ini Kalau Berniat Beli
                                               title  similarity
2  Begini Cara Mudah Bikin Tarikan Motor Matic Ho...    0.332244
1  Sering Disangka TPS Err

In [4]:
# Import library
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re
import numpy as np

# 1. Membaca dataset JSON
data = pd.read_json('berita.json')

# Menampilkan preview data
print("Preview data:")
print(data[['title']].head())  # Menampilkan hanya judul untuk ringkasan

# 2. Preprocessing
def preprocess_text(text):
    # Lowercasing
    text = text.lower()

    # Menghapus karakter non-alfabet
    text = re.sub(r'[^a-z\s]', '', text)

    # Tokenisasi
    tokens = text.split()

    # Stopword removal dan stemming
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Daftar stopword
    stopwords = set(["yang", "dan", "di", "ke", "dari", "pada", "untuk", "dengan"])
    tokens = [stemmer.stem(word) for word in tokens if word not in stopwords]

    # Gabungkan kembali token
    return ' '.join(tokens)

# Terapkan preprocessing pada kolom 'content'
data['processed_content'] = data['content'].apply(preprocess_text)

# 3. Vektorisasi dengan TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data['processed_content'])

# 4. Fungsi rekomendasi
def recommend_articles(input_title, data, tfidf_matrix, top_n=5):
    # Cari index artikel berdasarkan judul
    idx = data[data['title'] == input_title].index[0]

    # Hitung cosine similarity
    cosine_sim = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()

    # Urutkan berdasarkan skor similarity
    similar_indices = cosine_sim.argsort()[::-1][1:top_n+1]

    # Ambil artikel dengan skor tertinggi
    recommended = data.iloc[similar_indices].copy()
    recommended['similarity'] = cosine_sim[similar_indices]

    # Tambahkan filter kategori jika diperlukan
    category = data.iloc[idx]['category']
    recommended = recommended[recommended['category'] == category]

    return recommended

Preview data:
                                               title
0  Honda Vario 125 Lama Banyak Dicari, Waspada Pe...
1  Sering Disangka TPS Error, Ini Gejala Fuel Pum...
2  Begini Cara Mudah Bikin Tarikan Motor Matic Ho...
3  Aerox 155 dan Vario 125 Korban Tebas Gran Max ...
4  Lebih Murah, Subtitusi Rantai Keteng Vario 125...


In [6]:
# 5. Pilihan artikel
print("\nPilih artikel berdasarkan nomor:")
for i, title in enumerate(data['title']):
    print(f"{i + 1}. {title}")

# Ambil input dari pengguna
try:
    choice = int(input("\nMasukkan nomor artikel pilihan: ")) - 1
    if 0 <= choice < len(data):
        input_article = data.iloc[choice]['title']
        recommendations = recommend_articles(input_article, data, tfidf_matrix)
        print(f"\nRekomendasi untuk artikel: {input_article}")
        print(recommendations[['title', 'similarity']])
    else:
        print("Nomor yang dimasukkan tidak valid.")
except ValueError:
    print("Input harus berupa angka.")


Pilih artikel berdasarkan nomor:
1. Honda Vario 125 Lama Banyak Dicari, Waspada Penyakit Ini Kalau Berniat Beli
2. Sering Disangka TPS Error, Ini Gejala Fuel Pump Honda Vario 125 Lemah
3. Begini Cara Mudah Bikin Tarikan Motor Matic Honda Vario 125 Enteng
4. Aerox 155 dan Vario 125 Korban Tebas Gran Max Melintir, Denyut Nadi Mas Suraji Hilang
5. Lebih Murah, Subtitusi Rantai Keteng Vario 125 Bisa Pakai Merek Ini
6. Cara Rawat Standar Dua Honda Vario 125 Biar Enggak Gampang Macet
7. Mesin Sekelas Vario 125 tapi Lebih Garang dari Nmax, Ini Pesona Motor Matic 125cc Unik dari Yamaha
8. Media Vietnam Kaget dengan Gebrakan STY Pasca Dipecat PSSI: Tidak Terduga
9. Pasca Kim Jong-jin Blak-blakan, Giliran Asisten STY Lainnya Buka Suara
10. Mantan Asisten STY Ceritakan Reaksi STY Usai Terima Surat Pemecatan: Alasan STY Pilih Diam Jelang Kedatangan Kluivert
11. Tak Tahan Lagi, Shin Tae-yong Bongkar 1 Kebobrokan PSSI, Selama Ia 5 Tahun Jadi Pelatih di Timnas Indonesia, STY: Jujur...
12. Timnas Sat